In [90]:
import warnings
from selenium import webdriver
from selenium.webdriver.common.by import By
import glob
import os
import pandas as pd
import numpy as np
import math
import xlsxwriter as xw
from PIL import Image
import dropbox
import os


def download_file(username, password):
    driver = webdriver.Chrome(executable_path="C:\\Users\\SWCA\\Downloads\\chromedriver.exe")
    driver.get("https://login.microsoftonline.com/")
    driver.implicitly_wait(5)
    element = driver.find_element(By.XPATH,'//*[@id="i0116"]')
    element.click()
    element.send_keys(username)
    button = driver.find_element(By.XPATH,'//*[@id="idSIButton9"]')
    button.click()
    element = driver.find_element(By.XPATH,'//*[@id="i0118"]')
    element.send_keys(password)
    button = driver.find_element(By.XPATH,'/html/body/div/form[1]/div/div/div[2]/div[1]/div/div/div/div/div/div[3]/div/div[2]/div/div[4]/div[2]/div/div/div/div/input')
    button.click()
    button = driver.find_element(By.XPATH,'/html/body/div/form/div/div/div[2]/div[1]/div/div/div/div/div/div[3]/div/div[2]/div/div[3]/div[2]/div/div/div[1]/input')
    button.click()
    driver.implicitly_wait(2)
    driver.get("https://forms.office.com/Pages/DesignPage.aspx?auth_pvr=OrgId&auth_upn=durdaneta%40spinewellnessamerica.com#FormId=fyMOSlQm5UKkxUY8kPE2lwj6-aPCoslMvwftPFkfgX9UN0oxNVVZNUJGNE4yV0FHSFE4MkZJRkNSMy4u")
    button = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div[2]/div[1]/div[1]/div/div[2]')
    button.click()
    button = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div[2]/div[1]/div[2]/div/div/div[1]/div[3]/div/button')
    button.click()

def get_file():
    list_of_files = glob.glob('C:\\Users\\SWCA\Downloads\\*.xlsx')
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file
    
def create_dataframe(lastest_file):
    df = pd.read_excel(latest_file)
    df.columns = df.columns.str.replace(r'\d+', '')
    df = df.iloc[:,4:len(df.columns)]

    df2 = pd.concat([df['Name of Patient'].melt()['value'],df['Date of Birth'].melt()['value'],df['Diagnostic'].melt()['value'],df['Consult or Follow Up'].melt()['value'],df['Facesheet'].melt()['value']], axis=1, keys=['Name of Patient','Date of Birth','Diagnostic','Consult or Follow Up','Facesheet'])

    inx = np.array(np.where(df.columns.get_loc("Name of Patient"))).tolist()[0]
    val = len(df)
    for x in range(0,len(inx)):
        for y in range(0,val):
            try:
                if isinstance(df.iloc[y][inx[x+1]],str):
                    df = df.append(df.iloc[y])
            except IndexError:
                continue

    df = df.iloc[:,0:4].reset_index().iloc[:,1:len(df.columns)+1]
    df2 = df2.dropna(0,thresh=3).reset_index().iloc[:,1:len(df2.columns)+1]
    df = df.join(df2).sort_values("Physician").sort_values("Name").sort_values("Location").sort_values("Date of services provided").reset_index().iloc[:,1:len(df.columns)+len(df2.columns)+1]
    return df

def get_col_widths(dataframe):
    idx_max = max([len(str(s)) for s in dataframe.index.values] + [len(str(dataframe.index.name))])
    return [idx_max] + [max([len(str(s)) for s in dataframe[col].values] + [len(col)]) for col in dataframe.columns]


def create_and_upload(df, token)
    dbx = dropbox.Dropbox(token)
    dest_folder = f'/{doc} {loc} {time}'

    date_columns = ["Date of services provided","Date of Birth"]
    df[date_columns] = df[date_columns].astype(str)
    udoc = df[df.columns[1]].unique().tolist()
    uname = df[df.columns[0]].unique().tolist()
    utime = df[df.columns[2]].unique().tolist()
    uloc = df[df.columns[3]].unique().tolist()

    for name in uname:
        for doc in udoc:
            for loc in uloc:
                for time in utime:
                    test = df[(df[df.columns[0]]==name) & (df[df.columns[1]]==doc)& (df[df.columns[2]]==time) & (df[df.columns[3]]==loc)]
                    filename = 'C:\\Users\\SWCA\Downloads\\'+str(loc)+" "+str(time)+" "+str(name)+".xlsx"
                    fc_indexes = test[~df['Facesheet'].isnull()].index.tolist()
                    workbook = xw.Workbook(filename)
                    worksheet = workbook.add_worksheet()
                    head_form = workbook.add_format({'bold':True})
                    worksheet.set_row_pixels(0, 20,head_form)

                    for i, width in enumerate(get_col_widths(test)):
                        worksheet.set_column(i-1, i-1, width)

                    if len(test)>0:
                        for head in range(0,len(test.columns)):
                            worksheet.write(0, head,test.columns[head])

                        for x in range(0,len(test)):
                            for y in range(0,len(test.columns)-1):
                                try:
                                    worksheet.write(x+1, y,test.iloc[x,y])
                                except:
                                    pass

                        for index in fc_indexes:
                            try:
                                url = test.loc[index][8]
                                directory = url.replace("https://asjwc-my.sharepoint.com/personal/durdaneta_spinewellnessamerica_com/Documents/", r"C:\Users\SWCA\OneDrive - Spine and Wellness Centers of America/").replace("/",'\\').replace("%20"," ")
                                test2=test.reset_index()
                                location = str(test2[test2['Facesheet']==url].index[0]+2)
                                img = Image.open(directory)
                                width, height = img.size
                                if width>height:
                                    img = img.rotate(270,Image.NEAREST, expand=1)
                                    img.save(directory)
                                width, height = img.size
                                cellnum = 20
                                cells = height/cellnum
                                target = 20/cells
                                worksheet.insert_image('I'+location, directory,{'x_scale': target, 'y_scale': target})
                                form = workbook.add_format({'valign':'top'})
                                worksheet.set_row_pixels(int(location)-1, cellnum*20,form)
                            except:
                                pass

                        workbook.close()

                        try:
                            res = dbx.files_create_folder_v2(dest_folder)
                            with open(directory, "rb") as f:
                                dbx.files_upload(f.read(), f'{dest_folder}/{os.path.basename(directory)}', mute = True)
                        except:
                            with open(directory, "rb") as f:
                                dbx.files_upload(f.read(), f'{dest_folder}/{os.path.basename(directory)}', mute = True)

warnings.filterwarnings("ignore")

download_file('username','password')

token = 'token'

create_and_upload(create_dataframe(get_file()),token)

This script will collect, clean and distribute the operational data of medical practitioners for billing, payroll and auditing purposes. 